In [15]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [16]:
img_height = 180
img_width = 180
num_classes = 5

data_root_orig = "/home/yee/DataSet/flower_photos"
data_root = pathlib.Path(data_root_orig)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_root,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(180, 180),
    batch_size=32)

class_names = train_ds.class_names

data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal",
                                                     input_shape=(img_height,
                                                                  img_width,
                                                                  3)),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1),
    ]
)


def create_model():
    model = Sequential([
        data_augmentation,
        layers.experimental.preprocessing.Rescaling(1./255),
        layers.Conv2D(16, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(
                      from_logits=True),
                  metrics=['accuracy'])
    return model

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


In [17]:
# 创建模型实例
model = create_model()

# 恢复权重
model.load_weights('/home/yee/GraduationDesign/Model/checkpoints/Flowers/Flowers_checkpoint')

In [35]:
#正例
#data_root_orig_Yes = "/home/yee/DataSet/flower_photos/daisy/"
#data_root_orig_Yes = "/home/yee/DataSet/flower_photos/dandelion/"
#data_root_orig_Yes = "/home/yee/DataSet/flower_photos/roses/"
#data_root_orig_Yes = "/home/yee/DataSet/flower_photos/sunflowers/"
data_root_orig_Yes = "/home/yee/DataSet/flower_photos/tulips/"

data_root_Yes = pathlib.Path(data_root_orig_Yes)

#反例
data_root_orig_No="/home/yee/DataSet/TEST/"
data_root_No = pathlib.Path(data_root_orig_No)

#test_class="daisy"
#test_class="dandelion"
#test_class="roses"
#test_class="sunflowers"
test_class="tulips"

In [36]:
#正文件夹测试
TP=0
FN=0
numbers_test=0
# 遍历文件夹中的每个文件测试
for item in data_root_Yes.iterdir():
    # print(item)
    img = keras.preprocessing.image.load_img(
        item, target_size=(img_height, img_width) 
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    numbers_test+=1
    type_test=class_names[np.argmax(score)]
    confidence=100 * np.max(score)
    if type_test==test_class:
        TP+=1
    else:
        FN+=1
    #print("Belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score)))
print("numbers_test:{} TP:{} FN:{}".format(numbers_test,TP,FN))

numbers_test:799 TP:610 FN:189


In [37]:
#负文件夹测试
TN=0
FP=0
# 遍历文件夹中的每个文件测试
for item in data_root_No.iterdir():
    # print(item)
    img = keras.preprocessing.image.load_img(
        item, target_size=(img_height, img_width) 
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    numbers_test+=1
    type_test=class_names[np.argmax(score)]
    confidence=100 * np.max(score)
    if type_test==test_class:
        FP+=1
    else:
        TN+=1
    #print("Belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score)))
print("numbers_test:{} TN:{} FP:{}".format(numbers_test,TN,FP))

numbers_test:1599 TN:701 FP:99


In [38]:
precision=TP/(TP+FP)
recall=TP/(TP+FN)
F1=2*TP/(numbers_test+TP-TN)
print("P:{} R:{} F1:{}".format(precision,recall,F1))

P:0.8603667136812412 R:0.7634543178973717 F1:0.8090185676392573
